## Exercise 53 - inDegrees + Max

In [1]:
from graphframes import GraphFrame

inputPathVertexes = "/data/students/bigdata-01QYD/ex_data/Ex53/data/vertexes.csv"
inputPathEdges = "/data/students/bigdata-01QYD/ex_data/Ex53/data/edges.csv"
outputPath = "resOut_ex53/"

In [3]:
verticesDF = spark.read.load(inputPathVertexes, format="csv", header=True, inferSchema=True)
edgesDF = spark.read.load(inputPathEdges, format="csv", header=True, inferSchema=True)

In [4]:
verticesDF.printSchema()
verticesDF.show()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)

+---+-----+---+
| id| name|age|
+---+-----+---+
| u1|Alice| 34|
| u2|  Bob| 36|
| u3| John| 30|
| u4|David| 29|
| u5| Paul| 32|
| u6| Adel| 36|
| u7| Eddy| 60|
+---+-----+---+



In [5]:
edgesDF.printSchema()
edgesDF.show()

root
 |-- src: string (nullable = true)
 |-- dst: string (nullable = true)
 |-- linktype: string (nullable = true)

+---+---+--------+
|src|dst|linktype|
+---+---+--------+
| u1| u2|  friend|
| u1| u4|  friend|
| u1| u5|  friend|
| u2| u1|  friend|
| u2| u3|  follow|
| u3| u2|  follow|
| u4| u1|  friend|
| u4| u5|  friend|
| u5| u1|  friend|
| u5| u4|  friend|
| u5| u6|  follow|
| u6| u3|  follow|
| u7| u6|  follow|
+---+---+--------+



Consider only the users with at least one follower. Store in the output folder the user(s) with the maximum number of followers <br />
▪ One user per line <br />
▪ Format: user id, number of followers <br />

Use the CSV format to store the result

In [24]:
filteredEdges = edgesDF.filter("linktype = 'follow'")
g = GraphFrame(verticesDF, filteredEdges)

In [25]:
nFollowersDF = g.inDegrees.withColumnRenamed("inDegree","nFollowers")
nFollowersDF.show()

+---+----------+
| id|nFollowers|
+---+----------+
| u3|         2|
| u6|         2|
| u2|         1|
+---+----------+



In [26]:
# compute the "max" function
# it will return a DF with just one row and one column "max(nFollowers)"
# rename it into "maxFollowes"
# the the first (and the only one) row
# get the element from the column "maxFollowers"
maxFollowers = nFollowersDF.agg({"nFollowers" : "max"})\
                           .withColumnRenamed("max(nFollowers)","maxFollowers")\
                           .first()\
                           .maxFollowers

In [27]:
maxFollowers

2

In [28]:
# filter the rows from nFollowersDF that has 
# a number of followes equal to "maxFollowers"
filteredUsers = nFollowersDF.filter(nFollowersDF.nFollowers == maxFollowers)
filteredUsers.show()

+---+----------+
| id|nFollowers|
+---+----------+
| u3|         2|
| u6|         2|
+---+----------+



In [30]:
filteredUsers.write.csv(outputPath, header=True)